In [ ]:
from flask import Flask, request, Response, make_response
from enum import Enum
from yfunc import *

"""
drop table if exists fish;
create table fish (
    id integer PRIMARY KEY AUTOINCREMENT,
    identity varchar(64) NOT NULL,
    type varchar(16) NOT NULL,
    value blob DEFAULT NULL,
    description text NOT NULL DEFAULT '',
    tags varchar(128) NOT NULL DEFAULT '',
    is_marked tinyint NOT NUll DEFAULT 0,
    is_locked tinyint NOT NUll DEFAULT 0,
    extra_info text NOT NULL DEFAULT '{}',
    create_time DATETIME NOT NULL DEFAULT(datetime(CURRENT_TIMESTAMP, 'localtime')),
    update_time DATETIME NOT NULL DEFAULT(datetime(CURRENT_TIMESTAMP, 'localtime')),
    CONSTRAINT unique_data UNIQUE (identity)
);

create index index_time on fish (update_time);

drop TRIGGER if exists update_fish;
CREATE TRIGGER update_fish
AFTER UPDATE ON fish
FOR EACH ROW
BEGIN
    UPDATE fish SET update_time = datetime(CURRENT_TIMESTAMP, 'localtime') WHERE id = NEW.id;
END;

drop table if exists request;
create table request (
    id integer PRIMARY KEY AUTOINCREMENT,
    request_id varchar(64),
    url varchar(64),
    time_cost text,
    origin_input text,
    real_input text,
    response text,
    extra_info text,
    create_time DATETIME NOT NULL DEFAULT(datetime(CURRENT_TIMESTAMP, 'localtime'))
);




"""

In [ ]:
class Config:
    work_path = '/Users/bytedance/sol/tfdataservice'
    db_name = 'data.db'
    path__db = work_path + '/' + db_name
    path__resource = work_path + '/resource'
    path__log = work_path + '/log'
    max_size__fish_save_to_db = 10 * 1024 * 1024 # 10MB

In [ ]:
class FishType(Enum):
    text = 1
    image = 2

    @staticmethod
    def from_name(name: str) -> 'FishType':
        for e in FishType:
            if e.name == name:
                return e
        return None
    
class RespStatus(Enum):
    success = 'S'
    skip = 'P'
    fail = 'F'
    

In [ ]:
def init_logger():
    logger.remove(None)
    logger.add(lambda message: print(message))
    logger.add(Config.path__log + "/{time:YYYY-MM-DD}.log", rotation="00:00")

In [ ]:
def get_dict_resp(status: RespStatus, message: str, extra: str=''):
    return {
        'code': status.value + ''.join([s[0] for s in ystr(message).split(remove_null=True)]).upper() + extra,
        'status': status.name,
        'msg': message,
    }

In [ ]:
class Fish:

    def __init__(self, row: tuple) -> None:  
        self.id = row[0]  
        self.identity = row[1] 
        self.type = row[2] 
        if self.type == 'text':
            if isinstance(row[3], str):
                self.value = row[3]
            else:
                try:
                    self.value = ybytes(row[3]).to_str()
                except:
                    self.value = None
        else:
            self.value = None
        self.description = row[4] 
        self.tags = [] if row[5] == '' else row[5].split(',') 
        self.is_marked = True if row[6] == 1 else False
        self.is_locked = True if row[7] == 1 else False
        self.extra_info = row[8] 
        self.create_time = row[9] 
        self.update_time = row[10]
    
    @staticmethod
    def from_rows(rows: list[tuple]) -> list['Fish']:
        res = []
        for row in rows:
            try:
                res.append(Fish(row))
            except Exception as e:
                # todo: Note may print long bytes
                e.add_note(f"Note: row={row}")
                logger.exception(f'error when parse fish from db record, ignore record', e)
        return res

In [ ]:
class DB:

    fp = ystr(Config.path__db).filepath()

    @staticmethod
    def request__insert (
        request_id: str,
        url: str,
        time_cost: str,
        origin_input: str,
        real_input: str,
        response: str,
        extra_info: str
    ) -> None:
        DB.fp.db() \
            .table('request') \
            .row() \
            .field('request_id', request_id) \
            .field('url', url) \
            .field('time_cost', time_cost) \
            .field('origin_input', origin_input) \
            .field('real_input', real_input) \
            .field('response', response) \
            .field('extra_info', extra_info) \
            .insert()
        
    @staticmethod
    def request__update_time(request_id: str, time_cost: str) -> None:
        DB.fp.db() \
            .table('request') \
            .row() \
            .field('time_cost', time_cost) \
            .where(f"request_id = '{request_id}'") \
            .update()

    @staticmethod
    def fish__search (
            fuzzys: str = None,
            value: str = None,
            description: str = None,
            identity: str = None,
            type: list[str] = None,
            tags: list[str] = None,
            is_marked: int = None,
            is_locked: int = None,
            limit: int = None,
        ) -> list['Fish']:
        condition = '1=1'
        if fuzzys != None:
            condition += f" and (value like '%{fuzzys}%' or description like '%{fuzzys}%')"
        if value != None:
            condition += f" and (value = '{value}')"
        if description != None:
            condition += f" and (description = '{description}')"
        if identity != None:
            condition += f" and (identity = '{identity}')"
        if type != None:
            keyword = "'" + "','".join(type) + "'"
            condition += f" and (type in ({keyword}))"
        if tags != None:
            keyword = f"%{'%'.join(tags)}%"
            condition += f" and (tags like '{keyword}')"
        if is_marked != None:
            condition += f" and (is_marked = {is_marked})"
        if is_locked != None:
            condition += f" and (is_locked = {is_locked})"
        extra = 'order by update_time desc'
        if limit != None:
            extra += f" limit {limit}"
        res = DB.fp.db() \
            .table('fish') \
            .where(condition) \
            .extra(extra) \
            .select(print_sql=True)
        return Fish.from_rows(res)

    @staticmethod
    def fish__pick(id: int) -> list[Fish]:
        res = DB.fp.db() \
            .table('fish') \
            .where(f"id={id}") \
            .select(print_sql=True)
        return Fish.from_rows(res)
    
    @staticmethod
    def fish__exist(identity: str) -> bool:
        res = DB.fp.db().execute(f"select count(*) from fish where identity='{identity}';", print_sql=True)
        return res[0][0] > 0
    
    @staticmethod
    def fish__insert (
            value: bytes,
            description: str,
            identity: str,
            type: str,
            tags: str,
            extra_info: str,
        ) -> None:
        DB.fp.db() \
            .table('fish') \
            .row() \
            .field('value', ..., value) \
            .field('description', description) \
            .field('identity', identity) \
            .field('type', type) \
            .field('tags', tags) \
            .field('extra_info', extra_info) \
            .insert(print_sql=True)
        
    @staticmethod
    def fish__update (
            id: int,
            value: bytes = None,
            description: str = None,
            identity: str = None,
            type: str = None,
            tags: str = None,
            is_marked: int = None,
            is_locked: int = None,
            extra_info: str = None,
        ) -> None:
        DB.fp.db() \
            .table('fish') \
            .row() \
            .field('value', ..., value) \
            .field('description', description) \
            .field('identity', identity) \
            .field('type', type) \
            .field('tags', tags) \
            .field('is_marked', is_marked) \
            .field('is_locked', is_locked) \
            .field('extra_info', extra_info) \
            .where(f'id={id}') \
            .update(print_sql=True)
        
    @staticmethod
    def fish__delete(id: int) -> None:
        DB.fp.db().table('fish').where(f'id={id}').delete(print_sql=True)


In [ ]:
class Service():

    @staticmethod
    def add_request_log (
        request_id: str,
        url: str,
        time_cost: str,
        origin_input: dict,
        real_input: dict,
        response: dict,
        extra_info: str
    ) -> None:
        origin_input = ystr().json().from_object(origin_input)
        if 'value' in real_input:
            real_input['value'] = str(real_input['value'])
        real_input = ystr().json().from_object(real_input)
        response = ystr().json().from_object(response)
        DB.request__insert(request_id, url, time_cost, origin_input, real_input, response, extra_info)

    @staticmethod
    def update_request_time (
        request_id: str,
        time_cost_info: dict,
    ) -> None:
        DB.request__update_time(request_id, ystr().json().from_object(time_cost_info))
        
    @staticmethod
    def statistic() -> dict:
        res = DB.fish__search()
        type_stats = {}
        tag_stats = {}
        mark_stats = {'marked':0, 'unmarked':0}
        lock_stats = {'locked':0, 'unlocked':0}
        for r in res:
            if r.type in type_stats:
                type_stats[r.type] += 1
            else:
                type_stats[r.type] = 1
            for t in r.tags:
                if t in tag_stats:
                    tag_stats[t] += 1
                else:
                    tag_stats[t] = 1
            if r.is_marked:
                mark_stats['marked'] += 1
            else:
                mark_stats['unmarked'] += 1
            if r.is_locked:
                lock_stats['locked'] += 1
            else:
                lock_stats['unlocked'] += 1
        return {
            'total_count': len(res),
            'type': type_stats,
            'tag': tag_stats,
            'mark': mark_stats,
            'lock': lock_stats,
        }
            
    @staticmethod
    def search_fish (
            fuzzys: str = None, # like value or like desc
            value: str = None, # like
            description: str = None, # like
            identity: str = None,
            type: list[FishType] = None, # match any
            tags: list[str] = None, # match each
            is_marked: bool = None,
            is_locked: bool = None,
            limit: int = None,
        ) -> list['Fish']:
        if type != None:
            type = [e.name for e in type]
        if tags != None:
            tags = ylist(tags).unique().sort()
        if is_marked != None:
            is_marked = 1 if is_marked else 0
        if is_locked != None:
            is_locked = 1 if is_locked else 0
        return DB.fish__search(
            fuzzys=fuzzys, value=value, description=description, identity=identity,
            type=type, tags=tags, is_marked=is_marked, is_locked=is_locked, limit=limit,
        )
    
    @staticmethod
    def pick_fish(id) -> Fish:
        res = DB.fish__pick(id)
        if len(res) == 0:
            return None
        if len(res) > 1:
            logger.warning(f'fish(id={id}) duplicated in database, ignore others')
        return res[0]
    
    @staticmethod
    def add_fish (
            value: bytes,
            description: str,
            type: FishType,
            tags: list[str],
            extra_info: str,
        ) -> dict:
        if value == None:
            return get_dict_resp(RespStatus.fail, 'value can not be null', 'SVAF')
        if type == None:
            return get_dict_resp(RespStatus.fail, 'type can not be null or invalid', 'SVAF')
        if tags != None:
            tags = ylist(tags).unique().sort()
            tags = ','.join(tags)
        identity = ybytes(value).md5()
        if DB.fish__exist(identity):
            return get_dict_resp(RespStatus.fail, 'data duplicated', 'SVAF')
        # todo: check size and save file
        DB.fish__insert(
            value=value, description=description, identity=identity,
            type=type.name, tags=tags, extra_info=extra_info,
        )
        return get_dict_resp(RespStatus.success, 'success', 'SVAF')
    
    @staticmethod
    def remove_fish(identity: str) -> dict:
        res = DB.fish__search(identity=identity)
        if len(res) == 0:
            return get_dict_resp(RespStatus.skip, 'data not exists', 'SVRF')
        if res[0].is_locked:
            return get_dict_resp(RespStatus.fail, 'fish is locked', 'SVRF')
        DB.fish__delete(id=res[0].id)
        return get_dict_resp(RespStatus.success, 'success', 'SVRF')
    
    @staticmethod
    def mark_fish(identity: str) -> dict:
        res = DB.fish__search(identity=identity)
        if len(res) == 0:
            return get_dict_resp(RespStatus.fail, 'data not exists', 'SVMF')
        if res[0].is_locked:
            return get_dict_resp(RespStatus.fail, 'fish is locked', 'SVMF')
        if res[0].is_marked:
            return get_dict_resp(RespStatus.skip, 'fish has been marked', 'SVMF')
        DB.fish__update(id=res[0].id, is_marked=1)
        return get_dict_resp(RespStatus.success, 'success', 'SVMF')
    
    @staticmethod
    def unmark_fish(identity: str) -> dict:
        res = DB.fish__search(identity=identity)
        if len(res) == 0:
            return get_dict_resp(RespStatus.fail, 'data not exists', 'SVUMF')
        if res[0].is_locked:
            return get_dict_resp(RespStatus.fail, 'fish is locked', 'SVUMF')
        if not res[0].is_marked:
            return get_dict_resp(RespStatus.skip, 'fish is not marked', 'SVUMF')
        DB.fish__update(id=res[0].id, is_marked=0)
        return get_dict_resp(RespStatus.success, 'success', 'SVUMF')
    
    @staticmethod
    def lock_fish(identity: str) -> dict:
        res = DB.fish__search(identity=identity)
        if len(res) == 0:
            return get_dict_resp(RespStatus.fail, 'data not exists', 'SVLF')
        if res[0].is_locked:
            return get_dict_resp(RespStatus.skip, 'fish has been locked', 'SVLF')
        DB.fish__update(id=res[0].id, is_locked=1)
        return get_dict_resp(RespStatus.success, 'success', 'SVLF')
    
    @staticmethod
    def unlock_fish(identity: str) -> dict:
        res = DB.fish__search(identity=identity)
        if len(res) == 0:
            return get_dict_resp(RespStatus.fail, 'data not exists', 'SVULF')
        if not res[0].is_locked:
            return get_dict_resp(RespStatus.skip, 'fish is not locked', 'SVULF')
        DB.fish__update(id=res[0].id, is_locked=0)
        return get_dict_resp(RespStatus.success, 'success', 'SVULF')
    
    @staticmethod
    def pin_fish(identity: str) -> dict:
        res = DB.fish__search(identity=identity)
        if len(res) == 0:
            return get_dict_resp(RespStatus.fail, 'data not exists', 'SVPF')
            return {'code': -12, 'msg': 'fail: data not exists'}
        DB.fish__update(id=res[0].id, type=res[0].type)
        return get_dict_resp(RespStatus.fail, 'data not exists', 'SVPF')
        return {'code': 0, 'msg': 'success'}
    
    @staticmethod
    def modify_fish (
            identity: str, 
            description: str = None,
            tags: list[str] = None,
            extra_info: str = None,
    ) -> dict:
        if description == None and tags == None and extra_info == None:
            return {'code': 6, 'msg': 'skip: nothing to update'}
        res = DB.fish__search(identity=identity)
        if len(res) == 0:
            return {'code': -13, 'msg': 'fail: data not exists'}
        if res[0].is_locked:
            return {'code': -14, 'msg': 'fail: fish is locked'}
        if tags != None:
            tags = ylist(tags).unique().sort()
            tags = ','.join(tags)
        DB.fish__update(id=res[0].id, description=description, tags=tags, extra_info=extra_info)
        return {'code': 0, 'msg': 'success'}
    

In [ ]:
def control(func):
    def handle_input(func):
        input = {}
        lack_para = []
        dup_para = []
        import inspect
        func_paras = inspect.signature(func).parameters
        for func_para in func_paras:
            if func_para in request.args:
                if func_para not in input:
                    input[func_para] = request.args[func_para]
                else:
                    dup_para.append(func_para)
            if func_para in request.form:
                if func_para not in input:
                    input[func_para] = request.form[func_para]
                else:
                    dup_para.append(func_para)
            if func_para in request.files:
                if func_para not in input:
                    input[func_para] = ybytes(request.files[func_para].read())
                else:
                    dup_para.append(func_para)
            if func_para not in input and func_paras[func_para].default is inspect.Parameter.empty:
                lack_para.append(func_para)
        return input, lack_para, dup_para
    def handle_output(data, time_cost, status=200) -> Response:
        if type(data) == dict:
            res = {}
            res['time_cost'] = time_cost
            res['code'] = data.get('code', '0') 
            res['msg'] = data.get('msg', 'ok')
            if 'code' in data:
                del data['code']
            if 'msg' in data:
                del data['msg']
            if data == None or len(data) > 0:
                res['data'] = data
            res = ystr().json().from_object(res)
            return Response(res, status=status, content_type='application/json')
        return make_response(data)
    def wrap_func(*args, **kwargs):
        if len(args) > 0 or len(kwargs) > 0:
            logger.warning(f'warp func ignore para: args={args}, kwargs={kwargs}')
        request_id = ystr.uuid()
        logger.info(f"request start: request_id={request_id}, url={request.base_url}")
        try:
            t1 = int(time.time()*1000)
            input, lack_para, dup_para = handle_input(func)
            if len(lack_para) > 0:
                res = {'code':-899, 'msg':f'parameters required: {lack_para}'}
            elif len(dup_para) > 0:
                res = {'code':-888, 'msg':f'parameters duplicated: {dup_para}'}
            else:
                res = func(**input)
            t2 = int(time.time()*1000)
        except Exception as e:
            e.add_note(f'Note: url = {request.base_url}')
            logger.exception(e)
            res = {'code':-999, 'msg': str(e)}
            t2 = int(time.time()*1000)
        resp = handle_output(res, t2-t1)    
        t3 = int(time.time()*1000)
        Service.add_request_log(
            request_id=request_id, url=request.base_url, time_cost=f'{t3-t1}+:{t2-t1},{t3-t2}',
            origin_input = {
                'request.args': request.args,
                'request.form': request.form,
                'request.files': request.files,
            },
            real_input = input, 
            response = {
                'code': resp.status_code,
                'size': ybytes(resp.data).size(),
                'content_type': resp.content_type,
                'content_length': resp.content_length,
                'preview': ybytes(resp.data).desc()[:1000],
            },
            extra_info = '',
        )
        t4 = int(time.time()*1000)
        Service.update_request_time(request_id, {
            'total': t4-t1,
            'execute': t2-t1,
            'make_response': t3-t2,
            'record_log': t4-t3,            
        })
        logger.info(f"request end: request_id={request_id}, url={request.base_url}, time_cost={t4-t1}, input={input}, output={resp}")
        return resp
    wrap_func.__name__ = f'wrap__{func.__name__}'
    return wrap_func

In [ ]:
app = Flask(__name__)

@app.route('/', methods=['GET'])
@control
def hello() -> str:
    return ystr().timestamp().now()

@app.route('/stats', methods=['GET'])
@control
def stats() -> dict:
    return Service.statistic()
    
@app.route('/fish/search', methods=['GET'])
@control
def search_fish (
            fuzzys: str = None, 
            value: str = None, 
            description: str = None, 
            identity: str = None,
            type: str = None, 
            tags: str = None, 
            is_marked: str = None,
            is_locked: str = None,
            page: str = None,
    ) -> dict:
    if type != None:
        type = ylist(FishType.from_name(t) for t in type.split(',')).filter(FishType)
    if tags != None:
        tags = tags.split(',')
    if is_marked != None:
        if ystr(is_marked).of('true', '1'):
            is_marked = True
        elif ystr(is_marked).of('false', '0'):
            is_marked = False
        else:
            is_marked = None
    if is_locked != None:
        if ystr(is_locked).of('true', '1'):
            is_locked = True
        elif ystr(is_locked).of('false', '0'):
            is_locked = False
        else:
            is_locked = None
    if page != None:
        try:
            page = page.split(',')
            page_num = int(page[0]) if len(page) > 0 else 1
            page_size = int(page[1]) if len(page) > 1 else 20
        except:
            return {'code': 0, 'msg': 'success'}
        
        
    res = Service.search_fish(
        fuzzys=fuzzys, value=value, description=description, 
        identity=identity, type=type, tags=tags, 
        is_marked=is_marked, is_locked=is_locked, limit=limit,
    )
    return {
        'total_count': len(res),
        'fish': res,
    }

@app.route('/fish/pick', methods=['GET'])
@control
def pick_fish(id: str) -> dict:
    res = Service.pick_fish(id)
    return {
        'fish': res,
    }

@app.route('/fish/add', methods=['POST'])
@control
def add_fish (
        value: bytes,
        type: str,
        description: str = None,
        tags: str = None,
        extra_info: str = None,
    ) -> dict:
    if type != None:
        type = FishType.from_name(type)
    if tags != None:
        tags = tags.split(',')
    return Service.add_fish(
        value=value, description=description, type=type, 
        tags=tags, extra_info=extra_info,
    )

@app.route('/fish/modify', methods=['POST'])
@control
def modify_fish (
        identity: str, 
        description: str = None,
        tags: str = None,
        extra_info: str = None,
    ) -> dict:
    if tags != None:
        tags = tags.split(',')
    return Service.modify_fish(
        identity=identity, description=description,
        tags=tags, extra_info=extra_info,
    )
    
@app.route('/fish/pin', methods=['POST'])
@control
def pin_fish(identity: str) -> dict:
    return Service.pin_fish(identity)

@app.route('/fish/remove', methods=['POST'])
@control
def remove_fish(identity: str) -> dict:
    return Service.remove_fish(identity)

@app.route('/fish/lock', methods=['POST'])
@control
def lock_fish(identity: str) -> dict:
    return Service.lock_fish(identity)

@app.route('/fish/unlock', methods=['POST'])
@control
def unlock_fish(identity: str) -> dict:
    return Service.unlock_fish(identity)

@app.route('/fish/mark', methods=['POST'])
@control
def mark_fish(identity: str) -> dict:
    return Service.mark_fish(identity)

@app.route('/fish/unmark', methods=['POST'])
@control
def unmark_fish(identity: str) -> dict:
    return Service.unmark_fish(identity)


In [ ]:
init_logger()
app.run()